# Cluster of neighbouhoods in Toronto

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.22.0-pyh9f0ad1d_0

The following packages will be UPDATED:

  ca-certificates     pkgs/main::ca-certificates-2020.1.1-0 --> conda-forge::ca-certificates-2020.4.5.1-hecc5488_0

The following packages will be SUPERSEDED by a higher-priority channel:

  certifi    

In [23]:
df = pd.read_csv('/resources/labs/DP0701EN/finout.csv', keep_default_na=False, na_values=[""])
gc = pd.read_csv('Geospatial_Coordinates (1).csv')

In [24]:
df

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [25]:
gc.reset_index(drop=True, inplace=True)
new_stack = pd.concat([df, gc], axis=1)

In [26]:
new_stack

,PostalCode,Borough,Neighbourhood,PostalCode,Latitude,Longitude
0,M3A,North York,Parkwoods,M1B,43.806686,-79.194353
1,M4A,North York,Victoria Village,M1C,43.784535,-79.160497
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",M1E,43.763573,-79.188711
3,M6A,North York,"Lawrence Manor, Lawrence Heights",M1G,43.770992,-79.216917
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",M1H,43.773136,-79.239476
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",M1J,43.744734,-79.239476
6,M1B,Scarborough,"Malvern, Rouge",M1K,43.727929,-79.262029
7,M3B,North York,Don Mills,M1L,43.711112,-79.284577
8,M4B,East York,"Parkview Hill, Woodbine Gardens",M1M,43.716316,-79.239476
9,M5B,Downtown Toronto,"Garden District, Ryerson",M1N,43.692657,-79.264848


Above is the dataframe obtained from the previous parts. Now, we are going to simplify the dataframe, whose Bourough only contains the word Toronto. 

In [35]:
toronto_data = new_stack[new_stack['Borough'] == 'Toronto'].reset_index(drop=True)
toronto_data

,PostalCode,Borough,Neighbourhood,PostalCode,Latitude,Longitude


We realise if we only key in 'Toronto', there won't be any data. Now, let's try putting Downtown Toronto instead

In [41]:
dtoronto_data = new_stack[new_stack['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
dtoronto_data

,PostalCode,Borough,Neighbourhood,PostalCode,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",M1E,43.763573,-79.188711
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",M1H,43.773136,-79.239476
2,M5B,Downtown Toronto,"Garden District, Ryerson",M1N,43.692657,-79.264848
3,M5C,Downtown Toronto,St. James Town,M1W,43.799525,-79.318389
4,M5E,Downtown Toronto,Berczy Park,M2L,43.757490,-79.374714
5,M5G,Downtown Toronto,Central Bay Street,M2R,43.782736,-79.442259
6,M6G,Downtown Toronto,Christie,M3A,43.753259,-79.329656
7,M5H,Downtown Toronto,"Richmond, Adelaide, King",M3K,43.737473,-79.464763
8,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",M4C,43.695344,-79.318389
9,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",M4L,43.668999,-79.315572


Let's get the geographical coordinates for Downtown Toronto

In [39]:
address = 'Downtown Toronto, CA'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 43.6563221, -79.3809161.


In [44]:
# create map of downtown toronto using latitude and longitude values
map_dtoronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(dtoronto_data['Latitude'], dtoronto_data['Longitude'], dtoronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_dtoronto)  
    
map_dtoronto

In [45]:
CLIENT_ID = 'LZGGP3NZK4W3Q2IRUBZPORHTGOHLIAI4VAJRGK3VMBH3NG4G' # your Foursquare ID
CLIENT_SECRET = 'MYDPVGGOVQ3JQKKX51V4UFEE1CUS452VV1EYW5OLCPVXBDJX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LZGGP3NZK4W3Q2IRUBZPORHTGOHLIAI4VAJRGK3VMBH3NG4G
CLIENT_SECRET:MYDPVGGOVQ3JQKKX51V4UFEE1CUS452VV1EYW5OLCPVXBDJX


## Let's explore the first neighborhood.

In [46]:
dtoronto_data.loc[0, 'Neighbourhood']

'Regent Park, Harbourfront'

In [48]:
neighborhood_latitude = dtoronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = dtoronto_data.loc[0, 'Longitude'] # neighborhood longitude value
neighborhood_name = dtoronto_data.loc[0, 'Neighbourhood'] # neighborhood name
print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.7635726, -79.1887115.


In [49]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [50]:
results = requests.get(url).json()

In [51]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we'll convert the json data to a dataframe

In [52]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,RBC Royal Bank,Bank,43.766790,-79.191151
1,G & G Electronics,Electronics Store,43.765309,-79.191537
2,Big Bite Burrito,Mexican Restaurant,43.766299,-79.190720
3,Enterprise Rent-A-Car,Rental Car Location,43.764076,-79.193406
4,Woburn Medical Centre,Medical Center,43.766631,-79.192286


In [60]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [61]:
dtoronto_venues = getNearbyVenues(names=dtoronto_data['Neighbourhood'],
                                   latitudes=dtoronto_data['Latitude'],
                                   longitudes=dtoronto_data['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


In [62]:
dtoronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Berczy Park,2,2,2,2,2,2
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17,17,17,17,17,17
Central Bay Street,5,5,5,5,5,5
Christie,3,3,3,3,3,3
Church and Wellesley,8,8,8,8,8,8
"Commerce Court, Victoria Hotel",4,4,4,4,4,4
"First Canadian Place, Underground city",2,2,2,2,2,2
"Garden District, Ryerson",4,4,4,4,4,4
"Harbourfront East, Union Station, Toronto Islands",9,9,9,9,9,9


Now let's analyze each neighbourhood

In [64]:
dtoronto_onehot = pd.get_dummies(dtoronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dtoronto_onehot['Neighbourhood'] = dtoronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [dtoronto_onehot.columns[-1]] + list(dtoronto_onehot.columns[:-1])
dtoronto_onehot = dtoronto_onehot[fixed_columns]

dtoronto_onehot.head()

,Neighbourhood,Airport,Athletics & Sports,Auto Workshop,Bakery,Bank,Bar,Baseball Field,Beer Store,Bookstore,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Cafeteria,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Comic Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Curling Ice,Dance Studio,Dessert Shop,Diner,Discount Store,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Food,Food & Drink Shop,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gas Station,General Entertainment,Gourmet Shop,Grocery Store,Gym,Hakka Restaurant,Hardware Store,Home Service,Ice Cream Shop,Indie Movie Theater,Intersection,Italian Restaurant,Kids Store,Latin American Restaurant,Light Rail Station,Liquor Store,Martial Arts Dojo,Medical Center,Mexican Restaurant,Middle Eastern Restaurant,Movie Theater,Park,Pet Store,Pharmacy,Pizza Place,Playground,Pub,Rental Car Location,Restaurant,Sandwich Place,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Social Club,South American Restaurant,Spa,Steakhouse,Summer Camp,Supplement Shop,Sushi Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Thrift / Vintage Store,Vegetarian / Vegan Restaurant,Video Store,Wings Joint,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [65]:
dtoronto_grouped = dtoronto_onehot.groupby('Neighbourhood').mean().reset_index()

In [67]:
num_top_venues = 5

for hood in dtoronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = dtoronto_grouped[dtoronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0           Cafeteria   0.5
1   Martial Arts Dojo   0.5
2             Airport   0.0
3  Mexican Restaurant   0.0
4                 Pub   0.0


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0  Light Rail Station  0.12
1         Yoga Studio  0.06
2       Burrito Place  0.06
3       Garden Center  0.06
4              Garden  0.06


----Central Bay Street----
          venue  freq
0  Home Service   0.2
1          Bank   0.2
2   Pizza Place   0.2
3      Pharmacy   0.2
4   Coffee Shop   0.2


----Christie----
                venue  freq
0   Convenience Store  0.33
1                Park  0.33
2   Food & Drink Shop  0.33
3             Airport  0.00
4  Mexican Restaurant  0.00


----Church and Wellesley----
                       venue  freq
0                Pizza Place  0.25
1             Sandwich Place  0.12
2  Middle Eastern Restaurant  0.12
3       

In [68]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [69]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = dtoronto_grouped['Neighbourhood']

for ind in np.arange(dtoronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dtoronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Cafeteria,Martial Arts Dojo,Yoga Studio,Cosmetics Shop,Dance Studio,Dessert Shop,Diner,Discount Store,Electronics Store,Falafel Restaurant
1,"CN Tower, King and Spadina, Railway Lands, Har...",Light Rail Station,Yoga Studio,Spa,Garden,Fast Food Restaurant,Farmers Market,Park,Pizza Place,Restaurant,Skate Park
2,Central Bay Street,Pizza Place,Pharmacy,Bank,Home Service,Coffee Shop,Fast Food Restaurant,Dance Studio,Dessert Shop,Diner,Discount Store
3,Christie,Convenience Store,Park,Food & Drink Shop,Fish & Chips Shop,Dance Studio,Dessert Shop,Diner,Discount Store,Electronics Store,Falafel Restaurant
4,Church and Wellesley,Pizza Place,Sandwich Place,Coffee Shop,Chinese Restaurant,Intersection,Middle Eastern Restaurant,Discount Store,Yoga Studio,Falafel Restaurant,Dance Studio


### Now, we'll start to cluster the neighbourhood

In [70]:
# set number of clusters
kclusters = 5

dtoronto_grouped_clustering = dtoronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dtoronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 1, 1, 2, 1, 2, 4, 1, 1, 1], dtype=int32)

In [75]:
# add clustering labels
#neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

dtoronto_merged = dtoronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
dtoronto_merged = dtoronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

dtoronto_merged.head() 

,PostalCode,Borough,Neighbourhood,PostalCode,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",M1E,43.763573,-79.188711,1,Rental Car Location,Bank,Electronics Store,Mexican Restaurant,Intersection,Breakfast Spot,Medical Center,Yoga Studio,Fast Food Restaurant,Dessert Shop
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",M1H,43.773136,-79.239476,1,Athletics & Sports,Hakka Restaurant,Bakery,Thai Restaurant,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Yoga Studio,Fast Food Restaurant
2,M5B,Downtown Toronto,"Garden District, Ryerson",M1N,43.692657,-79.264848,1,General Entertainment,College Stadium,Skating Rink,Café,Yoga Studio,Fast Food Restaurant,Dance Studio,Dessert Shop,Diner,Discount Store
3,M5C,Downtown Toronto,St. James Town,M1W,43.799525,-79.318389,1,Fast Food Restaurant,Chinese Restaurant,Breakfast Spot,Grocery Store,Pharmacy,Thrift / Vintage Store,Pizza Place,Bank,Coffee Shop,Bubble Tea Shop
4,M5E,Downtown Toronto,Berczy Park,M2L,43.757490,-79.374714,0,Cafeteria,Martial Arts Dojo,Yoga Studio,Cosmetics Shop,Dance Studio,Dessert Shop,Diner,Discount Store,Electronics Store,Falafel Restaurant


In [77]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dtoronto_merged['Latitude'], dtoronto_merged['Longitude'], dtoronto_merged['Neighbourhood'], dtoronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters